# Static Replication of a Barrier Option

This notebook replicates the QuantLib `Replication` example: build a static
replicating portfolio of vanilla options that approximates a down-and-out
barrier option, following the approach outlined in Section 10.2 of Mark Joshi's
*The Concepts and Practice of Mathematical Finance*.

The key idea is to construct a portfolio of European puts whose value matches
the barrier option at expiry (final payoff) and along the barrier boundary
(knock-out condition). The more time points used along the barrier, the better
the approximation.

In [1]:
import pyquantlib as ql

print(f"PyQuantLib {ql.__version__} (QuantLib {ql.QL_VERSION})")

PyQuantLib 0.1.0 (QuantLib 1.40)


## Market Setup

In [2]:
today = ql.Date(29, 5, 2006)
ql.Settings.evaluationDate = today

# Option parameters
barrier = 70.0
rebate = 0.0
strike = 100.0
underlying_value = 100.0

# Market data (mutable quotes for scenario analysis)
underlying = ql.SimpleQuote(underlying_value)
risk_free_rate = ql.SimpleQuote(0.04)
volatility = ql.SimpleQuote(0.20)
maturity = today + ql.Period("1Y")

# Flat yield and vol curves
day_counter = ql.Actual365Fixed()
flat_rate = ql.FlatForward(0, ql.NullCalendar(), risk_free_rate, day_counter)
flat_vol = ql.BlackConstantVol(0, ql.NullCalendar(), volatility, day_counter)

# Black-Scholes process
process = ql.BlackScholesProcess(underlying, flat_rate, flat_vol)

print(f"Spot:       {underlying_value}")
print(f"Strike:     {strike}")
print(f"Barrier:    {barrier}")
print(f"Rate:       {risk_free_rate.value():.0%}")
print(f"Volatility: {volatility.value():.0%}")
print(f"Maturity:   {maturity}")

Spot:       100.0
Strike:     100.0
Barrier:    70.0
Rate:       4%
Volatility: 20%
Maturity:   May 29th, 2007


## Reference: Analytic Barrier Price

Price the down-and-out put using the closed-form `AnalyticBarrierEngine`.
This is the target value the replicating portfolios will try to match.

In [3]:
exercise = ql.EuropeanExercise(maturity)
payoff = ql.PlainVanillaPayoff(ql.Put, strike)

barrier_engine = ql.AnalyticBarrierEngine(process)
european_engine = ql.AnalyticEuropeanEngine(process)

reference_option = ql.BarrierOption(
    ql.BarrierType.DownOut, barrier, rebate, payoff, exercise,
)
reference_option.setPricingEngine(barrier_engine)

reference_value = reference_option.NPV()
print(f"Down-and-out barrier put NPV: {reference_value:.6f}")

Down-and-out barrier put NPV: 4.260726


## Building the Replicating Portfolio

The replication has two parts:

**Final payoff** (matching the barrier option at expiry if not knocked out):
- Long a put struck at K
- Short a digital (cash-or-nothing) put struck at B, notional K - B
- Short a put struck at B

**Barrier kills** (forcing portfolio value to zero along the barrier):
- At each time point, subtract a put struck at B with a notional chosen
  so the portfolio value is zero at (S=B, t).

More time points along the barrier produce a better approximation.

In [4]:
def make_european_put(payoff, exercise):
    """Create a European put with the analytic engine attached."""
    option = ql.VanillaOption(payoff, exercise)
    option.setPricingEngine(european_engine)
    return option


def build_base_portfolio():
    """Build the terminal payoff replication (common to all portfolios)."""
    portfolio = ql.CompositeInstrument()

    # Long put struck at K
    put_k = make_european_put(payoff, exercise)
    portfolio.add(put_k)

    # Short digital put struck at B, notional (K - B)
    digital_payoff = ql.CashOrNothingPayoff(ql.Put, barrier, 1.0)
    digital_put = make_european_put(digital_payoff, exercise)
    portfolio.subtract(digital_put, strike - barrier)

    # Short put struck at B
    lower_payoff = ql.PlainVanillaPayoff(ql.Put, barrier)
    put_b = make_european_put(lower_payoff, exercise)
    portfolio.subtract(put_b)

    return portfolio


def add_barrier_kills(portfolio, dates_spec):
    """Kill portfolio value along the barrier at specified time points.

    dates_spec: list of (inner_maturity_period, kill_date_period) tuples
    """
    for mat_period, kill_period in dates_spec:
        inner_exercise = ql.EuropeanExercise(today + mat_period)
        inner_payoff = ql.PlainVanillaPayoff(ql.Put, barrier)
        put_n = make_european_put(inner_payoff, inner_exercise)

        # Move to (B, t) to evaluate
        ql.Settings.evaluationDate = today + kill_period
        underlying.setValue(barrier)

        notional = portfolio.NPV() / put_n.NPV()
        portfolio.subtract(put_n, notional)

    # Restore market conditions
    ql.Settings.evaluationDate = today
    underlying.setValue(underlying_value)

    return portfolio

In [5]:
# Portfolio 1: 12 monthly dates
monthly_dates = [
    (ql.Period(i, ql.Months), ql.Period(i - 1, ql.Months))
    for i in range(12, 0, -1)
]
portfolio_12 = add_barrier_kills(build_base_portfolio(), monthly_dates)

# Portfolio 2: 26 biweekly dates
biweekly_dates = [
    (ql.Period(i, ql.Weeks), ql.Period(i - 2, ql.Weeks))
    for i in range(52, 1, -2)
]
portfolio_26 = add_barrier_kills(build_base_portfolio(), biweekly_dates)

# Portfolio 3: 52 weekly dates
weekly_dates = [
    (ql.Period(i, ql.Weeks), ql.Period(i - 1, ql.Weeks))
    for i in range(52, 0, -1)
]
portfolio_52 = add_barrier_kills(build_base_portfolio(), weekly_dates)

## Results

Compare the three replicating portfolios against the analytic barrier price
under different market conditions.

In [6]:
def show_results(title, spot):
    """Display replication results for a given spot level."""
    underlying.setValue(spot)
    ref = reference_option.NPV()

    print(f"\n{'=' * 65}")
    print(f"{title} (S = {spot})")
    print(f"{'=' * 65}")
    print(f"{'Option':<40s} {'NPV':>10s} {'Error':>12s}")
    print(f"{'-' * 65}")
    print(f"{'Original barrier option':<40s} {ref:10.6f} {'N/A':>12s}")

    for label, port in [
        ("Replicating portfolio (12 dates)", portfolio_12),
        ("Replicating portfolio (26 dates)", portfolio_26),
        ("Replicating portfolio (52 dates)", portfolio_52),
    ]:
        val = port.NPV()
        print(f"{label:<40s} {val:10.6f} {val - ref:12.6f}")


show_results("Initial market conditions", underlying_value)
show_results("Out of the money", 110.0)
show_results("In the money", 90.0)

# Restore
underlying.setValue(underlying_value)
None


Initial market conditions (S = 100.0)
Option                                          NPV        Error
-----------------------------------------------------------------
Original barrier option                    4.260726          N/A
Replicating portfolio (12 dates)           4.322358     0.061632
Replicating portfolio (26 dates)           4.295464     0.034738
Replicating portfolio (52 dates)           4.280909     0.020183

Out of the money (S = 110.0)
Option                                          NPV        Error
-----------------------------------------------------------------
Original barrier option                    2.513058          N/A
Replicating portfolio (12 dates)           2.539365     0.026307
Replicating portfolio (26 dates)           2.528362     0.015304
Replicating portfolio (52 dates)           2.522105     0.009047

In the money (S = 90.0)
Option                                          NPV        Error
-----------------------------------------------------------

As the number of barrier kill dates increases (12 -> 26 -> 52), the
replication error shrinks. In the limit of continuous monitoring, the
static portfolio would perfectly replicate the barrier option.

The replication is most accurate when the spot is far from the barrier.
Near the barrier (in-the-money scenario), approximation quality depends
more heavily on the density of kill dates.